In [ ]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display, clear_output, HTML


import resourcecode
from resourcecode.multivariate_extremes.nataf import (
    censgaussfit,
    get_gpd_parameters,
    run_simulation,
    huseby,
)

In [ ]:
client = resourcecode.Client()
data = None

output_text = widgets.Output()
output_plots = widgets.Output()


def update_dataframe(change_url):
    print(change_url)
    global data
    with output_text:
        clear_output()
        print("The new selection is {new}".format(**change_url))
        print("Will re-compute the data end rerender the graphs")

    data = client.get_dataframe_from_url(
        change_url["new"], parameters=("hs", "ucur", "vcur")
    )
    update_plots()


def update_plots(event=None):
    print(data)
    with output_plots:
        clear_output()
        display(HTML("<h1>Multivariate Extremes analyses</h1>"))

        if data is None:
            display("You must provide a selection first")
            return
        display(plot_nataf(quantile_widget.value))

In [ ]:
url_widget = widgets.Text(
    value="",
    placeholder="Paste the URL of your selection",
    description="Selection:",
    disabled=False,
    continuous_update=False,
)
url_widget.observe(update_dataframe, "value")

quantile_widget = widgets.FloatSlider(
    description="Quantile:",
    min=0.1,
    max=0.99,
    value=0.8,
    step=0.01,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
quantile_widget.observe(update_plots, "value")
selection_widgets = widgets.VBox([url_widget, quantile_widget])


display(selection_widgets)
display(output_text)
display(output_plots)

In [ ]:
def plot_nataf(quant):
    gauss = censgaussfit(data.values, quant).x
    gpd_parameters = get_gpd_parameters(data, quant)
    nataf = run_simulation(gauss, quant, gpd_parameters)
    levels = np.array([0.9, 0.95, 0.975])

    X, Y, Z, theta = huseby(nataf, levels, 120)

    fig = go.Figure()

    fig.add_traces(
        [
            go.Scatter3d(
                x=X[:, i],
                y=Y[:, i],
                z=Z[:, i],
                mode="lines",
                name=level,
            )
            for i, level in enumerate(levels)
        ]
    )

    fig.update_layout(
        scene=dict(
            xaxis_title=data.columns[0],
            yaxis_title=data.columns[1],
            zaxis_title=data.columns[2],
        ),
        height=1200,
    )

    return fig